**sample code for hackathon with your data in directory** (by hchoi@handong.edu) 

put class directories in ./train/, ./val/, and ./test/.

for example, with 'dog', 'cat' classes
* ./train/dog/*.jpg
* ./train/cat/*.jpg
* ./val/dog/*.jpg
* ./val/cat/*.jpg
* ./test/dog/*.jpg
* ./test/cat/*.jpg

**submit your teamX.model and teamX.transform files. **

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os
import warnings
warnings.filterwarnings("ignore")

**try to change the network architecture**

In [0]:
class MyCNN(nn.Module):
    def __init__(self, input_size=32, output_dim=10):
        super(MyCNN,self).__init__()

        self.input_size=input_size # 100 x 100
        self.output_dim=output_dim

        self.layer = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 64 x (50 x 50)

            nn.Conv2d(64,128,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 128 x (25 x 25)
            
            nn.Conv2d(128,256,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 256 x (12 x 12)

            nn.Conv2d(256,128,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 128 x (6 x 6) 
            
            nn.Conv2d(128,64,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2) # 32 x (3 x 3) 
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(32*3*3,100),
            nn.ReLU(),
            nn.Linear(100,output_dim)
        )       
        
    def forward(self,x):
        batch_size, c, h, w = x.data.size()
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out


**try to change learning rate and input_size**

* be carefule with the input size, you have to change the network architecture accordingly

In [0]:
learning_rate = 0.0005
input_size=(100, 100)
output_dim=2

model = MyCNN(input_size=input_size, output_dim=output_dim).cuda()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**try to change transform function**

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3])
    ]),
}
test_transform = data_transforms['val']
torch.save(test_transform, 'drive/My Drive/public/results/teamX.transform')

**data loader**

In [0]:
batch_size = 64
data_dir = './drive/My Drive/public/data/toy_dog_cat'

image_datasets = {x: datasets.ImageFolder('./drive/My Drive/public/data/toy_dog_cat/'+x,
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                              shuffle=True, num_workers=4)
                  for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(class_names) 
print(f'Train image size: {dataset_sizes["train"]}')
print(f'Validation image size: {dataset_sizes["val"]}')

FileNotFoundError: ignored

**training**

In [0]:
num_epoch = 10

if not os.path.exists('drive/My Drive/public/results'):
    os.mkdir('drive/My Drive/public/results')    
    
for i in range(num_epoch):
    for j, [image,label] in enumerate(dataloaders['train']):
        x = image.cuda()
        y_= label.cuda()
        
        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 100 == 0:
            print(loss.data,i,j)
    
    with torch.no_grad(): 
      hits = 0
      for k,[image,label] in enumerate(dataloaders['val']):
          x = image.cuda()
          y_= label.cuda()
        
          output = model(x)
          y_est = output.argmax(1)
          hits = hits + sum(y_est == y_)
      print('hits, accuracy', hits, hits/(dataset_sizes["val"]+0.0))    
    torch.save(model,'drive/My Drive/public/results/teamX.model')
            
#param_list = list(model.children())
#print(param_list)


**TA will check your results based on teamX.model and teamX.transform as below**

In [0]:
test_batch_size = 4
model = torch.load('drive/My Drive/public/results/teamX.model')
test_transform = torch.load('drive/My Drive/public/results/teamX.transform')
model.cuda()
model.eval()
  
image_testset = datasets.ImageFolder('./drive/My Drive/public/data/toy_dog_cat/test',
                                          test_transform)
              
testloader = torch.utils.data.DataLoader(image_testset, batch_size=test_batch_size,
                                          shuffle=False, num_workers=4)

hits = 0
for k,[image,label] in enumerate(testloader):
    x = image.cuda()
    y_= label.cuda()
  
    output = model(x)
    print(output.argmax(1), label)
    y_est = output.argmax(1)
    hits = hits + sum(y_est == y_)
print('hits, accuracy', hits, hits/(len(image_testset)+0.0))

The end! (hchoi@handong.edu)